In [59]:
#### This code is written by anant to create codes for selecting based on following factors
################ 1. Number of micro clusters it fills
################ 2. Ensuring filling maximum number of cluster with minimum products
################ 3. Ensuring that substitutability data is taken into account while ensuring the products addition
import sqlalchemy as sqla  #this help us to execute sql statements from the python environment only

import getpass # it help us get password in * format. otherwise password is visible to everyone

import pandas as pd ## data wrangling functions in python

import numpy as np ## numpy for arrays manipulation 

# get date range of period specified by user input
# pull basket_ids & corresponding card_ids from 10pct for these dates
# join basket segs to above on basket id
# join customer segs to above on customer id

user_period = 201639
# # store username & password for connection in variable to be used later on
username = raw_input("Enter your Userid: ")    ### raw input is used to asked for entering values on cammand prompt
password = getpass.getpass("Enter your Password: ") ### mask the password using getpasss

## create engine is the starting point of any sqlalachemy connection.its a lazy intialisation. That it will establish connection
## only when connect or execute methods are invoked 
## keep a single engine for one application can allow concurrent execution
engine = sqla.create_engine('oracle://%s[AN_RT_WS18]:%s@exa_gbtukprdd_an' % (username, password)) 
### Import the required libraries
import pandas as pd
import numpy as np


#### Import the csv containing the products & componenets created in micro cluster 0090 codes
pdbsk = pd.read_csv('prod_cross_components.csv')

pdbsk['count_new'] = pdbsk.sum( axis = 1) - pdbsk.cluster


#### Read the sas datasets which provide data for substitution
sls = pd.read_sas('sls_output_201709_dh_insight.sas7bdat')

pdbsk.head()

sls.head()

Enter your Userid: ashwinik
Enter your Password: ········


,original_tpnb,sub_tpnb,sub_rank,times_picked,times_rejected,rejection_rate,SLS_process
0,50001758.0,63879027.0,1.0,NaN,NaN,NaN,0.0
1,50001758.0,80503332.0,2.0,NaN,NaN,NaN,0.0
2,50001758.0,62362415.0,3.0,NaN,NaN,NaN,0.0
3,50001758.0,62362486.0,4.0,NaN,NaN,NaN,0.0
4,50001758.0,62362496.0,5.0,NaN,NaN,NaN,0.0


In [61]:
#### Create a list of all columns into python
prod_list_columns = pdbsk.iloc[:,1:1270].columns.tolist()

### Convert the list to datafarme
prod_list = pd.DataFrame(prod_list_columns)

### Rename the column to be used for join
prod_list = prod_list.rename(columns={0: 'tpn'})

#### convert the tpnb from object datatype to float data-type
prod_list['tpn'] = pd.to_numeric(prod_list['tpn'], errors='coerce')


In [62]:
#### Join on the original products
sls1 = pd.merge(sls, prod_list,  how='inner', left_on=['original_tpnb'], right_on = ['tpn'])

#### Join on the substitute products
sls2 = pd.merge(sls1, prod_list,  how='inner', left_on=['sub_tpnb'], right_on = ['tpn'])

sls2.head(5)

,original_tpnb,sub_tpnb,sub_rank,times_picked,times_rejected,rejection_rate,SLS_process,tpn_x,tpn_y
0,5.00204e+07,5.1628e+07,2.0,89.0,15.0,0.168539,1.0,50020430,51627961
1,5.00205e+07,5.1628e+07,1.0,283.0,16.0,0.056537,1.0,50020453,51627961
2,5.00205e+07,5.1628e+07,6.0,66.0,27.0,0.409091,1.0,50020522,51627961
3,5.05048e+07,5.1628e+07,5.0,31.0,15.0,0.483871,1.0,50504780,51627961
4,5.05496e+07,5.1628e+07,2.0,376.0,44.0,0.117021,1.0,50549611,51627961


In [63]:
#### Create a list of which contains all the products 
cls_f = prod_list_columns

In [64]:
### Create a nummpy array with all zero values
x0 = np.zeros((1269,1269), int)

### Convert  the numpy array to the dataframe
x1 = pd.DataFrame(x0)

### Rename the columns to all the products available
x1.columns = cls_f

### Add the products in a new columns
x1['tpnb'] = cls_f

### Convert the wide data to long data
x2 = pd.DataFrame(x1.set_index('tpnb').unstack()).reset_index()

#### convert the tpnb from object datatype to float data-type
x2['level_0'] = pd.to_numeric(x2['level_0'], errors='coerce')
x2['tpnb'] = pd.to_numeric(x2['tpnb'], errors='coerce')

### Merge the data with the sls file 
x3 = pd.merge(x2, sls,  how='left', left_on=['level_0','tpnb'], right_on = ['original_tpnb','sub_tpnb'])

print x3.head()

x4 = x3.groupby(['level_0','tpnb'])['rejection_rate'].max().unstack()

x4.fillna(0,inplace = True)

    level_0      tpnb  0  original_tpnb    sub_tpnb  sub_rank  times_picked  \
0  50020430  50020430  0            NaN         NaN       NaN           NaN   
1  50020430  50020453  0     50020430.0  50020453.0       3.0        1419.0   
2  50020430  50020516  0     50020430.0  50020516.0       8.0           NaN   
3  50020430  50020522  0     50020430.0  50020522.0      12.0           NaN   
4  50020430  50020793  0            NaN         NaN       NaN           NaN   

   times_rejected  rejection_rate  SLS_process  
0             NaN             NaN          NaN  
1            70.0        0.049331          1.0  
2             NaN             NaN          0.0  
3             NaN             NaN          0.0  
4             NaN             NaN          NaN  


In [65]:
### convert the dataframe to the array
x5 = np.array(x4)

### Replace all the pairs which don't have rejection rate with 1
x5[x5 == 0] = 1

### Create a new data frame with copy of x5
x6 = pd.DataFrame(x5)

### rename all the columns in new dataframe with the products tpnbs
x6.columns = cls_f

### Add a new column which contain the product tpnb 
x6['tpnb'] = cls_f

### Create the index file similar to the source
x6.index = x4.index

### print the dataset 
x6.head(5)

,50020430,50020453,50020516,50020522,50020793,50020804,50020925,50021014,50021158,50021319,...,82545092,82545123,82629606,82629612,82629629,82629635,82629641,82629658,82629664,tpnb
level_0,,,,,,,,,,,,,,,,,,,,,
50020430,1.000000,0.049331,1.000000,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50020430
50020453,0.016149,1.000000,1.000000,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50020453
50020516,0.117647,1.000000,1.000000,0.040385,1.0,1.000000,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50020516
50020522,0.211268,0.188649,0.021532,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50020522
50020793,0.125000,1.000000,1.000000,1.000000,1.0,0.051479,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50020793


In [66]:
### import the package to required the time 
import time

# start_time = time.time()
### create a copy of products rejection rate pairs 
df = pdbsk.copy()

### Sort the values based on number of basket it belongs to 
df.sort_values('count_new',ascending = False, inplace = True)



In [67]:
# # Creating a function to scale the count of baskets for subs by (1 - rejection_rate)

# def f(row):
#     if row['count'] > 0:
#         val = row['count']*(1 - row['rejection_rate'])
#     else:
#         val = row['count']
#     return val

# start_time = time.time()

# prod = []


# # filtering for missions that have a certain no of baskets 
# ### parameters to be modified
# df2 = df[df['count_new'] > 3000] 

# df2.reset_index(inplace = True)

# df2.drop('index',inplace = True, axis = 1)

 


In [68]:


# # Looping through the dataframe mission by mission and product by product inside a mission 

# for i, row in df2.iterrows():
    
#     if i ==0:
    
#         Y = pd.DataFrame(row.iloc[1:1270]) # Selecting the first mission in the dataframe based on # of baskets
        

    
#         Y['tpnb'] = Y.index

#         Y.columns = ['count','tpnb']

#         Y.sort_values('count', ascending = False, inplace = True)

#         max_bsk = Y['count'].iloc[0]

#         tpn_list = Y['tpnb'].iloc[0]

#         prod.append(np.array(Y['tpnb'].iloc[0]))
    
#         dedup = np.array(pd.DataFrame(prod).drop_duplicates())

#         y2 = np.array(x6[dedup.ravel()])
    
#         z2 = np.array(df2.ix[:,1:1270])

#         for n in range(len(dedup)):
    
#             z3 = np.multiply((y2.T[n]),z2)
        
#             z2 = z3

#         X = pd.DataFrame(z2[i])    
        
#         X['tpnb'] = cls_f

#         X.columns = ['count','tpnb']

#         X.sort_values('count', ascending = False, inplace = True)

#         rng = (X['count']/max_bsk > 0.01).sum() # Looping only uptill count of baskets is greater than 1% of max # of baskets

#         for x in range(1,(rng+1)):
            
#             dedup = np.array(pd.DataFrame(prod).drop_duplicates())

#             y2 = np.array(x6[dedup.ravel()])
    
#             for n in range(len(dedup)):
    
#                 z4 = np.multiply((y2.T[n]),z2)
        
#                 z2 = z4

#             X = pd.DataFrame(z2[i])    
        
#             X['tpnb'] = cls_f

#             X.columns = ['count','tpnb']

#             X.sort_values('count', ascending = False, inplace = True)            
            
#             Y_sub = X.iloc[x:]
                            
#             Y_sub['ratio'] = Y_sub['count']/max_bsk

#             if Y_sub['ratio'].iloc[0] > 0.01: # Selecting the products that satisfy the 1% criteria
                
#                 prod.append(np.array(Y_sub['tpnb'].iloc[0]))

#     else:
    
#         Y = pd.DataFrame(z2[i])    
        
#         Y['tpnb'] = cls_f

#         Y.columns = ['count','tpnb']

#         Y.sort_values('count', ascending = False, inplace = True)

#         max_bsk = Y['count'].iloc[0]

#         tpn_list = Y['tpnb'].iloc[0]

#         prod.append(np.array(Y['tpnb'].iloc[0]))
    
#         dedup = np.array(pd.DataFrame(prod).drop_duplicates())

#         y2 = np.array(x6[dedup.ravel()])
    
#         for n in range(len(dedup)):
    
#             z3 = np.multiply((y2.T[n]),z2)
        
#             z2 = z3

#         X = pd.DataFrame(z2[i])    
        
#         X['tpnb'] = cls_f

#         X.columns = ['count','tpnb']

#         X.sort_values('count', ascending = False, inplace = True)

#         rng = (X['count']/max_bsk > 0.01).sum() # Looping only uptill count of baskets is greater than 1% of max # of baskets

#         for x in range(1,(rng+1)):
            
#             dedup = np.array(pd.DataFrame(prod).drop_duplicates())

#             y2 = np.array(x6[dedup.ravel()])
    
#             for n in range(len(dedup)):
    
#                 z4 = np.multiply((y2.T[n]),z2)
        
#                 z2 = z4

#             X = pd.DataFrame(z2[i])    
        
#             X['tpnb'] = cls_f

#             X.columns = ['count','tpnb']

#             X.sort_values('count', ascending = False, inplace = True)            
            
#             Y_sub = X.iloc[x:]
                            
#             Y_sub['ratio'] = Y_sub['count']/max_bsk

#             if Y_sub['ratio'].iloc[0] > 0.01: # Selecting the products that satisfy the 1% criteria
                
#                 prod.append(np.array(Y_sub['tpnb'].iloc[0]))
    
# print 'Running time : ' + str(time.time() - start_time) + 'seconds'   

In [69]:
# L = pd.DataFrame(prod).drop_duplicates()

# L.to_csv('exp_results_3k_n2.csv')

In [70]:
#  Creating a function to scale the count of baskets for subs by (1 - rejection_rate)

def f(row):
    if row['count'] > 0:
        val = row['count']*(1 - row['rejection_rate'])
    else:
        val = row['count']
    return val

start_time = time.time()

prod = []

In [71]:
cut_offs_test = [4000]

for j in cut_offs_test :

    # filtering for missions that have a certain no of baskets 

    df2 = df[df['count_new'] > j] 

    df2.reset_index(inplace = True)

    df2.drop('index',inplace = True, axis = 1)
    
    print 'The input considered for %s cutoffs' % (j)
    print df2.shape

# # # cls_f = df2.tpn

    # Looping through the dataframe mission by mission and product by product inside a mission 
    prod = []

    for i, row in df2.iterrows():

        if i ==0:

            Y = pd.DataFrame(row.iloc[1:1270]) # Selecting the first mission in the dataframe based on # of baskets

            Y['tpnb'] = Y.index

    #     ### ADDED by ASHWINI TO CONVERT THE dtypes to int
    #         Y['tpnb'] = Y['tpnb'].astype(int)
            Y.columns = ['count','tpnb']

            Y.sort_values('count', ascending = False, inplace = True)

    #         print Y

            max_bsk = Y['count'].iloc[0]

            tpn_list = Y['tpnb'].iloc[0]

            prod.append(np.array(Y['tpnb'].iloc[0]))

    #         print prod

            dedup = np.array(pd.DataFrame(prod).drop_duplicates())
    #         dedup = dedup.astype(int)
    #         print dedup

            y2 = np.array(x6[dedup.ravel()])

            z2 = np.array(df2.ix[:,1:1270])

            n = len(dedup) - 1

            z3 = np.multiply((y2.T[n]),z2)

            z2 = z3

            X = pd.DataFrame(z2[i])    

            X['tpnb'] = cls_f

            X.columns = ['count','tpnb']

            X.sort_values('count', ascending = False, inplace = True)

            rng = (X['count']/max_bsk > 0.01).sum() # Looping only uptill count of baskets is greater than 1% of max # of baskets

            for x in range(1,(rng+1)):

                dedup = np.array(pd.DataFrame(prod).drop_duplicates())

    #             dedup = dedup.astype(int)
    # #             print dedup

                y2 = np.array(x6[dedup.ravel()])

                n = len(dedup) - 1

                z4 = np.multiply((y2.T[n]),z2)

                z2 = z4

                X = pd.DataFrame(z2[i])    

                X['tpnb'] = cls_f

                X.columns = ['count','tpnb']

                X.sort_values('count', ascending = False, inplace = True)            

                Y_sub = X.iloc[x:]

                Y_sub['ratio'] = Y_sub['count']/max_bsk

                if Y_sub['ratio'].iloc[0] > 0.01: # Selecting the products that satisfy the 1% criteria

                    prod.append(np.array(Y_sub['tpnb'].iloc[0]))

        else:

            Y = pd.DataFrame(z2[i])   
    #         print 'running'

            Y['tpnb'] = cls_f
        ### ADDED by ASHWINI TO CONVERT THE dtypes to int
    #         Y['tpnb'] = Y['tpnb'].astype(int)
            Y.columns = ['count','tpnb']

            Y.sort_values('count', ascending = False, inplace = True)

            max_bsk = Y['count'].iloc[0]

            tpn_list = Y['tpnb'].iloc[0]

            prod.append(np.array(Y['tpnb'].iloc[0]))

            dedup = np.array(pd.DataFrame(prod).drop_duplicates())
    #         dedup = dedup.astype(int)
    #         print dedup

            y2 = np.array(x6[dedup.ravel()])

            n = len(dedup) - 1

            z3 = np.multiply((y2.T[n]),z2)

            z2 = z3

            X = pd.DataFrame(z2[i])    

            X['tpnb'] = cls_f

            X.columns = ['count','tpnb']

            X.sort_values('count', ascending = False, inplace = True)

            rng = (X['count']/max_bsk > 0.01).sum() # Looping only uptill count of baskets is greater than 1% of max # of baskets

            for x in range(1,(rng+1)):

                dedup = np.array(pd.DataFrame(prod).drop_duplicates())
    #             dedup = dedup.astype(int)

                y2 = np.array(x6[dedup.ravel()])

                n = len(dedup) - 1

                z4 = np.multiply((y2.T[n]),z2)

                z2 = z4

                X = pd.DataFrame(z2[i])    

                X['tpnb'] = cls_f

                X.columns = ['count','tpnb']

                X.sort_values('count', ascending = False, inplace = True)            

                Y_sub = X.iloc[x:]

                Y_sub['ratio'] = Y_sub['count']/max_bsk

                if Y_sub['ratio'].iloc[0] > 0.01: # Selecting the products that satisfy the 1% criteria

                    prod.append(np.array(Y_sub['tpnb'].iloc[0]))

    print 'Running time : ' + str(time.time() - start_time) + 'seconds' 

    print 'Running for %s cutoffs as follows' % (j)
    L = pd.DataFrame(prod).drop_duplicates()

    print 'Number of Products Selected for this iteration is'
    print L.shape[0]

    print 'Percentage Basket Coverage for this iterations is'
    tot_baskets = pdbsk['count_new'].sum()
    bask_covered = df2['count_new'].sum()
    print (float(bask_covered) /tot_baskets) * 100.00

/usr/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python2.7/site-packages/ipykernel/__main__.py:51: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
/usr/lib/python2.7/site-packages/ipykernel/__main__.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python2.7/site-packages/ipykernel/__main__.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy o

The input considered for 4000 cutoffs
(262, 1271)
Running time : 295.634449959seconds
Running for 4000 cutoffs as follows
Number of Products Selected for this iteration is
169
Percentage Basket Coverage for this iterations is
90.9411926348


In [72]:
# cls = row.iloc[1:1270].index

L = pd.DataFrame(prod).drop_duplicates()

L.to_csv('exp_results_3k_n2.csv')

In [73]:
#### This code is written by anant to create codes for selecting based on following factors
################ 1. Number of micro clusters it fills
################ 2. Ensuring filling maximum number of cluster with minimum products
################ 3. Ensuring that substitutability data is taken into account while ensuring the products addition

### Import the required libraries
import pandas as pd
import numpy as np

#### Read the sas datasets which provide data for substitution
sls = pd.read_sas('sls_output_201709_dh_insight.sas7bdat')

pdbsk.head()

sls.head()


,original_tpnb,sub_tpnb,sub_rank,times_picked,times_rejected,rejection_rate,SLS_process
0,50001758.0,63879027.0,1.0,NaN,NaN,NaN,0.0
1,50001758.0,80503332.0,2.0,NaN,NaN,NaN,0.0
2,50001758.0,62362415.0,3.0,NaN,NaN,NaN,0.0
3,50001758.0,62362486.0,4.0,NaN,NaN,NaN,0.0
4,50001758.0,62362496.0,5.0,NaN,NaN,NaN,0.0


In [74]:
### Apllt the filter to remove all products which have more than 20% rejection rate
sls_subs = sls[(sls.rejection_rate< 0.2) & (sls.rejection_rate > 0.0)]

In [75]:
#### Import the csv containing the products & componenets created in micro cluster 0090 codes
pdbsk = pd.read_csv('exp_results_3k_n2.csv')

In [76]:
#### Drop a garbage columns
pdbsk = pd.DataFrame(pdbsk.drop('Unnamed: 0',1))

In [77]:
#### rename the column to required data
pdbsk = pdbsk.rename(columns= {'0': 'tpn'})

In [78]:
#### Join on the original products
sls1 = pd.merge(sls, pdbsk,  how='inner', left_on=['original_tpnb'], right_on = ['tpn'])

In [79]:
#### Remove the null introduced
sls_nulls = sls1[ (sls1.rejection_rate > 0.0)]

In [80]:
### Create a combined list for both the subs & original tpnbs
list_cols = [0,1]
prod_subs = []
for i in list_cols:
    prods = sls_nulls.iloc[:,i]
    prod_subs.append(prods)

In [81]:
# cls = row.iloc[1:1270].index
#### Cobvert the each element of list to the column & make sure that they have the name 

prod_subs_transpose = pd.DataFrame(prod_subs[0]).drop_duplicates()
prod_subs_transpose1 = pd.DataFrame(prod_subs[1]).drop_duplicates()

prod_subs_transpose1 = prod_subs_transpose1.rename(columns= {'sub_tpnb': 'original_tpnb'})

In [82]:
### Create a final dataset to have all the products 
prod_subs_final =prod_subs_transpose.append([prod_subs_transpose1])

In [83]:
## Transfer the file to the required_location
prod_subs_final.to_sql('tpn_seleceted_subs',engine, 
                      if_exists = 'replace', 
                      index = False, 
                      dtype={'TPNB':sqla.types.INTEGER()
                             })

In [84]:
## SQL CALCULATIONS TO CREATE FILES REQUIRED
try:
    engine.execute('drop table ak_list')
except:
    print 'table does not exist '

engine.execute('create table ak_list as select a.* , b.original_tpnb as tpnb_selected  from AK_EXP_TRANS_test a left join\
            tpn_seleceted_subs b on a.tpnb = b.original_tpnb')




try:
    engine.execute('drop table ak_flag')
except:
    print 'table does not exist '

engine.execute("""
create table ak_flag as \
select a.*, case when tpnb_selected is null then 'not_included' else 'included' end as include_flag from ak_list a
""")

try:
    engine.execute('drop table ak_sales')
except:
    print 'table does not exist '

engine.execute("""create table ak_sales as \
select * from  \
(SELECT  transaction_fid,items,include_flag \
    FROM( \
select transaction_fid,include_flag,sum(spend_amt) as items from ak_flag group by transaction_fid,include_flag \
) a )  \
PIVOT  \
(\
sum(items) \
for include_flag in ('included' as include,'not_included' as not_included)\
)""")

try:
    engine.execute('drop table ak_sales_not_null')
except:
    print 'table does not exist '

engine.execute('create table ak_sales_not_null as \
select transaction_fid,NVL(include,0) as included_sales, NVL(not_included,0) as not_included_sales from ak_sales')

try:
    engine.execute('drop table ak_final')
except:
    print 'table does not exist '

engine.execute('create table ak_final as select sales_covered,count(transaction_fid) as transactions from \
(select ROUND(included_sales/ (included_sales+not_included_sales),2) as sales_covered ,transaction_fid from \
ak_sales_not_null) a group by sales_covered')

In [ ]:
sales_covered = pd.read_sql('select * from ak_final ',engine)

In [57]:
### Create Bins in the data based on certain values ####
bins = [-0.01, 0.25, 0.5, 0.75, 1]

### Group Category names Sales Covered###
group_names = ['below_25_percent','between_25_50_percent','between_50_75_percent','between_75_100_percent']

### Create the bins for response ###
sales_covered['coverage_bins'] = pd.cut(sales_covered['sales_covered'], bins, labels=group_names)

In [58]:
### print the summary of particular variable ###
freq_counts = sales_covered.groupby(['coverage_bins'])['transactions'].sum().to_frame().reset_index()

### add %age columns

freq_counts['Percentage_of_Variable'] = (freq_counts['transactions'] / freq_counts['transactions'].sum() ) * 100

print freq_counts

### For 1000 cutoff
#             coverage_bins  transactions  Percentage_of_Variable
# 0        below_25_percent        175100                5.476253
# 1   between_25_50_percent         35770                1.118707
# 2   between_50_75_percent         81594                2.551852
# 3  between_75_100_percent       2904978               90.853188

### For 0 cutoff
#             coverage_bins  transactions  Percentage_of_Variable
# 0        below_25_percent        138029                4.316857
# 1   between_25_50_percent         22069                0.690208
# 2   between_50_75_percent         64027                2.002444
# 3  between_75_100_percent       2973317               92.990491

### For 2000 cutoff
#             coverage_bins  transactions  Percentage_of_Variable
# 0        below_25_percent        181803                5.685889
# 1   between_25_50_percent         37883                1.184791
# 2   between_50_75_percent         83462                2.610274
# 3  between_75_100_percent       2894294               90.519046

            coverage_bins  transactions  Percentage_of_Variable
0        below_25_percent        181803                5.685889
1   between_25_50_percent         37883                1.184791
2   between_50_75_percent         83462                2.610274
3  between_75_100_percent       2894294               90.519046


In [29]:
sales_covered

,sales_covered,transactions,coverage_bins
0,1.00,2909001,between_75_100_percent
1,0.50,2374,between_25_50_percent
2,0.90,3285,between_75_100_percent
3,0.92,2250,between_75_100_percent
4,0.19,132,below_25_percent
5,0.15,50,below_25_percent
6,0.85,2537,between_75_100_percent
7,0.35,571,between_25_50_percent
8,0.11,22,below_25_percent
9,0.97,622,between_75_100_percent
